In [79]:
import os
import json
from elasticsearch import Elasticsearch

In [80]:
with open('../credentials.json') as f:
    credentials = json.load(f)

bonsai_url = credentials['bonsai_url']
access_key = credentials['access_key']
access_secret = credentials['access_secret']


FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [3]:
es = Elasticsearch(
    [bonsai_url],
    basic_auth=(access_key, access_secret),
    headers={'Content-Type': 'application/json'}
)
info = es.info()
version_number = info['version']['number']

print(f"Elasticsearch version: {version_number}")

Elasticsearch version: 7.2.0


In [7]:
terms = [7, 3]

# Construct the script source as a string
script_source = """
int count = 0;
for (long skill_id : doc['skill_ids'].values) {
  if (params.terms.contains(skill_id)) {
    count++;
  }
}
return count / (double) doc['skill_ids'].length;
"""

# Construct the query using the terms variable and script_source
query = {
  "query": {
    "term": {
      "language_ids": 1
    }
  }
}

In [75]:
terms = [1,2,3,4,5,6,8]

# Construct the script source as a string
script_source = """
double count = 0.0; 
for (long term : params.terms) { 
  if (doc['skill_ids'].contains(term)) { 
    count++; 
    } 
  } 
return count/ (double) params.terms.length;
"""

# Construct the query using the terms variable and script_source
query = {
  "query": {
    "function_score": {
      "query": {
        "terms": {
          "skill_ids": terms
        }
      },
      "functions": [
        {
          "script_score": {
            "script": {
              "source": script_source,
              "params": {
                "terms": terms
              }
            }
          }
        }
      ]
    }
  }
}

In [76]:
index_name = "learning_opportunities_production_20230711123025421"

In [77]:
response = es.search(index=index_name, body=query)

In [78]:
for elem in response['hits']['hits']:
    print(elem['_source']['course'], elem['_source']['skill_ids'], elem['_score'])

Training from the BACK of the Room Practitioner – Virtual Edition, Deutsch/Mai [8, 12, 7, 5, 1, 2, 3] 0.71428573
Training from the BACK of the Room Practitioner - Virtual Edition, English [3, 2, 1, 5, 7, 12, 8] 0.71428573
Training from the BACK of the Room! (TBR) Practitioner class, 2 Tage Präsenzkurs [3, 2, 1, 5, 7, 12, 8] 0.71428573
Training from the BACK of the Room Practitioner – Virtual Edition, Deutsch [3, 2, 1, 5, 7, 12, 8] 0.71428573
Sustainable investment: FFPC session [2, 6, 1, 5, 3, 9] 0.71428573
CAS Diskurskompetenzen für Führungskräfte [2, 5, 4, 3, 8, 7] 0.71428573
CAS Performance und Compensation Management [4, 5, 6, 7, 8, 9, 10] 0.5714286
CAS HR-Marketing [1, 3, 6, 8, 9] 0.5714286
CAS Human Capital & Leadership [1, 3, 4, 7, 8, 10, 11] 0.5714286
CAS Consulting, Communication & Culture [2, 3, 5, 8] 0.5714286
